In [14]:
# Section 8: Materialized Views
'''
This section focuses on Materialized Views.
 * A Materialized View is a physically stored table that contains the result set of a query, whereas a normal view is not stored physically 
 on disc.
 * This section will teach you about Materialized Views and highlight the main benefits.
'''

'\nThis section focuses on Materialized Views.\n * A Materialized View is a physically stored table that contains the result set of a query, whereas a normal view is not stored physically \n on disc.\n * This section will teach you about Materialized Views and highlight the main benefits.\n'

In [15]:
# Quick Recap on Views
'''
 - pdf
'''

'\n - pdf\n'

In [28]:
# What are Materialized Views
'''
SELECT emp.* ,dept.dname from emp join dept on emp.deptno = dept.deptno; --cost is 6

create materialized view my_emp_dept
AS 
SELECT emp.* ,dept.dname from emp join dept on emp.deptno = dept.deptno;

select * from my_emp_dept; -- cost is 3

create view v_emp_dept_t
AS 
SELECT emp.* ,dept.dname from emp join dept on emp.deptno = dept.deptno;

select * from v_emp_dept_t; --cost is 6

'''

'\nSELECT emp.* ,dept.dname from emp join dept on emp.deptno = dept.deptno; --cost is 6\n\ncreate materialized view my_emp_dept\nAS \nSELECT emp.* ,dept.dname from emp join dept on emp.deptno = dept.deptno;\n\nselect * from my_emp_dept; -- cost is 3\n\ncreate view v_emp_dept_t\nAS \nSELECT emp.* ,dept.dname from emp join dept on emp.deptno = dept.deptno;\n\nselect * from v_emp_dept_t; --cost is 6\n\n'

In [16]:
# Syntax
'''
 CREATE MATERIALIZED VIEW view-name
 BUILD
'''

'\n'

In [29]:
# Simple And Complex Materialized Views
'''
  - pdf
'''

'\n  - pdf\n'

In [31]:
# Refresh Types
'''
create materialized view mv_emp_deptno10
build immediate
refresh fast 
on commit 
disable query rewrite 
as 
select * from emp where deptno=10;

create materialized view log on emp;
select * from all_mview_logs;

drop materialized view mv_emp_deptno10;
drop materialized view  log on emp;
'''

# refresh complete 
'''
create materialized view mv_emp_deptno10
build immediate
refresh complete 
on commit 
disable query rewrite 
as 
select * from emp where deptno=10;



'''
# refresh force 
'''
create materialized view mv_emp_deptno10
build immediate
refresh force 
on commit 
disable query rewrite 
as 
select * from emp where deptno=10;
'''

# select * from all_snapshots;

'\ncreate materialized view mv_emp_deptno10\nbuild immediate\nrefresh complete \non commit \ndisable query rewrite \nas \nselect * from emp where deptno=10;\n\n\n\n'

In [32]:
# On Commit / On Demand
'''
create materialized view mv_emp_deptno10
build immediate
refresh force 
on commit 
disable query rewrite 
as 
select * from emp where deptno=10;

select * from mv_emp_deptno10;


insert into emp values(999,'TEsT','TEST',null,sysdate,1000,0,10);
commit;
select * from emp;

'''

#### 
'''
drop materialized view mv_emp_deptno10;
create materialized view mv_emp_deptno10
build immediate
refresh force 
on demand 
disable query rewrite 
as 
select * from emp where deptno=10;

select * from mv_emp_deptno10;
insert into emp values(888,'TEST','TEST',null,sysdate,1000,0,10);
select * from emp;

-- manually refresh or scedule it 
'''

####  Scedule #####
'''
drop materialized view mv_emp_deptno10;
create materialized view mv_emp_deptno10
build immediate
refresh force 
on demand 
start with sysdate next (sysdate + 1/1440)
disable query rewrite 
as 
select * from emp where deptno=10;

select * from mv_emp_deptno10;
insert into emp values(777,'TEST','TEST',null,sysdate,1000,0,10);
select * from emp;

-- manually refresh or scedule it 
'''

"\ncreate materialized view mv_emp_deptno10\nbuild immediate\nrefresh force \non commit \ndisable query rewrite \nas \nselect * from emp where deptno=10;\n\nselect * from mv_emp_deptno10;\n\n\ninsert into emp values(999,'TEsT','TEST',null,sysdate,1000,0,10);\ncommit;\nselect * from emp;\n\n"

In [33]:
# Manually Refresing Your Materialzied View
'''
drop materialized view mv_emp_deptno10;
create materialized view mv_emp_deptno10
build immediate
refresh complete 
on demand 
disable query rewrite 
as 
select * from emp where sal>3000;

select * from mv_emp_deptno10;
insert into emp values(888,'TEST','TEST',null,sysdate,4000,0,10);
select * from emp;
delete from emp where ename like 'TE%'

begin dbms_mview.refresh('mv_emp_deptno10', 'C'); end;

'''

"\ndrop materialized view mv_emp_deptno10;\ncreate materialized view mv_emp_deptno10\nbuild immediate\nrefresh complete \non demand \ndisable query rewrite \nas \nselect * from emp where sal>3000;\n\nselect * from mv_emp_deptno10;\ninsert into emp values(888,'TEST','TEST',null,sysdate,4000,0,10);\nselect * from emp;\ndelete from emp where ename like 'TE%'\n\nbegin dbms_mview.refresh('mv_emp_deptno10', 'C'); end;\n\n"

In [34]:
# Build Deferred
'''
drop materialized view mv_def;
create materialized view mv_def
build deferred
refresh force 
on demand 
start with (sysdate + 1/1440)
as 
select * from emp;

select * from mv_def;
'''

'\ndrop materialized view mv_def;\ncreate materialized view mv_def\nbuild deferred\nrefresh force \non demand \nstart with (sysdate + 1/1440)\nas \nselect * from emp;\n\nselect * from mv_def;\n'

In [35]:
# Query Rewrite
'''
drop materialized view mv_eqr;
create materialized view mv_eqr
build immediate
refresh force 
on commit 
enable query rewrite
as 
select * from emp where sal > 2000;
'''

'\ndrop materialized view mv_eqr;\ncreate materialized view mv_eqr\nbuild immediate\nrefresh force \non commit \nenable query rewrite\nas \nselect * from emp where sal > 2000;\n'

In [36]:
# Converting An Existing Table into A Materialied View
'''
 create table empdep as select emp.* ,dept.dname from emp left join dept on emp.deptno = dept.deptno where 1 = 2;
select * from empdep;


create materialized view empdep
on prebuilt table 
refresh complete 
on commit 
as 
select emp.*, dept.dname from emp join dept on emp.deptno = dept.deptno;


select * from empdep;

drop table empdep; -- must use DROP MATERIALIZED VIEW to drop 
drop materialized view empdep;
drop table empdep;

'''

'\n create table empdep as select emp.* ,dept.dname from emp left join dept on emp.deptno = dept.deptno where 1 = 2;\nselect * from empdep;\n\n\ncreate materialized view empdep\non prebuilt table \nrefresh complete \non commit \nas \nselect emp.*, dept.dname from emp join dept on emp.deptno = dept.deptno;\n\n\nselect * from empdep;\n\ndrop table empdep; -- must use DROP MATERIALIZED VIEW to drop \ndrop materialized view empdep;\ndrop table empdep;\n\n'

In [37]:
# Additional Guidance
'''
drop materialized view mv_join;

create materialized view mv_join
build immediate
refresh complete 
on commit
disable query rewrite 
as 
-- select emp.*, dept.dname from emp left join dept on emp.deptno = dept.deptno; -- cannot set the ON COMMIT refresh attribute for the materialized view
select emp.*, dept.dname from emp inner join dept on emp.deptno = dept.deptno;


create materialized view mv_join
build immediate
refresh complete 
on demand
disable query rewrite 
as 
select emp.*, dept.dname from emp left join dept on emp.deptno = dept.deptno; 

'''

'\ndrop materialized view mv_join;\n\ncreate materialized view mv_join\nbuild immediate\nrefresh complete \non commit\ndisable query rewrite \nas \n-- select emp.*, dept.dname from emp left join dept on emp.deptno = dept.deptno; -- cannot set the ON COMMIT refresh attribute for the materialized view\nselect emp.*, dept.dname from emp inner join dept on emp.deptno = dept.deptno;\n\n\ncreate materialized view mv_join\nbuild immediate\nrefresh complete \non demand\ndisable query rewrite \nas \nselect emp.*, dept.dname from emp left join dept on emp.deptno = dept.deptno; \n\n'

In [27]:
# Useful Links an resources
'''
'''

'\n'